In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import TensorDataset, Dataset, DataLoader

from dataclasses import dataclass
import math 

## GPT

In [ ]:
@dataclass
class ModelConfig:
    block_size: int = 50 # length of the input sequences
    vocab_size: int = 1  # time series dimensionality
    n_layer: int = 2
    n_embd: int = 8
    n_head: int = 4

class NewGELU(nn.Module):
    """
    Implementation of the GELU activation function currently in Google BERT repo (identical to OpenAI GPT).
    Reference: Gaussian Error Linear Units (GELU) paper: https://arxiv.org/abs/1606.08415
    """
    def forward(self, x):
        return 0.5 * x * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * torch.pow(x, 3.0))))

class CausalSelfAttention(nn.Module):
    """
    A vanilla multi-head masked self-attention layer with a projection at the end.
    It is possible to use torch.nn.MultiheadAttention here but I am including an
    explicit implementation here to show that there is nothing too scary here.
    """

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # causal mask to ensure that attention is only applied to the left in the input sequence
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                     .view(1, 1, config.block_size, config.block_size))
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k ,v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.c_proj(y)
        return y

class Block(nn.Module):
    """ an unassuming Transformer block """

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = nn.ModuleDict(dict(
            c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd),
            c_proj  = nn.Linear(4 * config.n_embd, config.n_embd),
            act     = NewGELU(),
        ))
        m = self.mlp
        self.mlpf = lambda x: m.c_proj(m.act(m.c_fc(x))) # MLP forward

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlpf(self.ln_2(x))
        return x

class Transformer(nn.Module):
    """ Transformer Language Model, exactly as seen in GPT-2 """

    def __init__(self, config):
        super().__init__()
        self.block_size = config.block_size

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Linear(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # report number of parameters (note we don't count the decoder parameters in lm_head)
        n_params = sum(p.numel() for p in self.transformer.parameters())
        print("number of parameters: %.2fk" % (n_params/1e3,))

    def get_block_size(self):
        return self.block_size

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.block_size, f"Cannot forward sequence of length {t}, block size is only {self.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device).unsqueeze(0) # shape (1, t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx.unsqueeze(-1)) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (1, t, n_embd)
        x = tok_emb + pos_emb
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x).squeeze(-1)

        # if we are given some desired targets also calculate the loss
        loss = None
        if targets is not None:
            loss = F.mse_loss(logits,targets)
        return logits, loss

In [ ]:
model = Transformer(ModelConfig())

number of parameters: 2.18k


## time series modelling

In [ ]:
import pandas as pd

In [ ]:
path = "data/AEP_hourly.csv"

In [ ]:
df = pd.read_csv(path)

In [ ]:
ts = df["AEP_MW"].to_numpy(dtype="float32")

In [ ]:
ts_train = ts[:100000]
ts_test = ts[100000:]

In [ ]:
class TS(Dataset):
    def __init__(self,ts):
        super().__init__()
        self.ts = ts
    def __len__(self):
        return len(self.ts)-50
    def __getitem__(self,idx):
        x = self.ts[idx:idx+50]
        y = self.ts[idx+1:idx+51] 
        return x,y

In [ ]:
data = TS(ts_train)
loader = DataLoader(data,batch_size=8, shuffle=True)
adam  = torch.optim.AdamW(model.parameters())
for _ in range(5):
    for i,batch in enumerate(loader):
        X,Y = batch
        model.zero_grad()
        logits, loss = model(X,Y)
        loss.backward()
        adam.step()
        if (i+1)%500==0:
            print(f"batch {i}: train loss {loss.item()}")

batch 499: train loss 150321120.0
batch 999: train loss 182423840.0
batch 1499: train loss 164693472.0
batch 1999: train loss 136400480.0
batch 2499: train loss 125565208.0
batch 2999: train loss 104868904.0
batch 3499: train loss 133050736.0
batch 3999: train loss 113600448.0
batch 4499: train loss 123169528.0
batch 4999: train loss 116370864.0
batch 5499: train loss 94413856.0
batch 5999: train loss 106890096.0
batch 6499: train loss 113709920.0
batch 6999: train loss 88361824.0
batch 7499: train loss 73270776.0
batch 7999: train loss 72086280.0
batch 8499: train loss 84046464.0
batch 8999: train loss 83209264.0
batch 9499: train loss 70266000.0
batch 9999: train loss 59385228.0
batch 10499: train loss 71721872.0
batch 10999: train loss 50568536.0
batch 11499: train loss 50120736.0
batch 11999: train loss 36134780.0
batch 499: train loss 41485992.0
batch 999: train loss 43334872.0
batch 1499: train loss 44712500.0
batch 1999: train loss 31564812.0
batch 2499: train loss 44551392.0
ba

In [ ]:
test_data = TS(ts_test)
test_loader = DataLoader(test_data, batch_size=8)

with torch.inference_mode():
    losses = []
    for i,batch in enumerate(test_loader):
        X,Y = batch
        logits, loss = model(X,Y)
        losses.append(loss.item())
        
    test_loss = torch.tensor(losses).mean().item()

In [ ]:
test_loss**.5 # RMSE

448.35325428171035

In [ ]:
x,y = test_data[0]
context = torch.tensor(x[:25]).view(1,-1) #  1,t

In [ ]:
context.shape

torch.Size([1, 25])

In [ ]:
logits, loss =model(context)
logits[:,[-1]].shape

torch.Size([1, 1])

In [ ]:
torch.cat((context,logits[:,[-1]]),-1).shape

torch.Size([1, 26])

In [ ]:
idx = torch.randint(len(test_data)-50,(1,))
x,y = test_data[idx]
context = torch.tensor(x[:25]).view(1,-1) #  1,t
ahead = 1
for _ in range(ahead):
    logits, loss =model(context)
    context = torch.cat((context,logits[:,[-1]]),-1)

In [ ]:
F.mse_loss(context.squeeze(0)[25:],torch.tensor(x[25:25+ahead])).item()**0.5

370.40526791610296